In [ ]:
import numpy as np
import pandas as pd
import uproot
import xgboost as xgb
import matplotlib.pyplot as plt
from xgbo import XgboRegressor
import os
from scipy.stats import norm
import matplotlib.mlab as mlab
import iminuit
import probfit
import probfit.pdf

In [ ]:
def load_data(file_name, entrystop, isEE=False):

    root_file = uproot.open(file_name)

    # The branches we need for the regression
    branches_EB = [ 'clusterRawEnergy', 'full5x5_e3x3', 'full5x5_eMax',
            'full5x5_e2nd', 'full5x5_eTop', 'full5x5_eBottom', 'full5x5_eLeft',
            'full5x5_eRight', 'full5x5_e2x5Max', 'full5x5_e2x5Top',
            'full5x5_e2x5Bottom', 'full5x5_e2x5Left', 'full5x5_e2x5Right',
            'full5x5_e5x5', 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
            'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
            'full5x5_sigmaIphiIphi', 'iEtaSeed', 'iPhiSeed', 'iEtaMod5',
            'iPhiMod2', 'iEtaMod20', 'iPhiMod20', 'genEnergy']

    branches_EE = [ 'clusterRawEnergy', 'full5x5_e3x3', 'full5x5_eMax',
            'full5x5_e2nd', 'full5x5_eTop', 'full5x5_eBottom', 'full5x5_eLeft',
            'full5x5_eRight', 'full5x5_e2x5Max', 'full5x5_e2x5Top',
            'full5x5_e2x5Bottom', 'full5x5_e2x5Left', 'full5x5_e2x5Right',
            'full5x5_e5x5', 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
            'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
            'full5x5_sigmaIphiIphi',
            'genEnergy', 'iXSeed', 'iYSeed', 'preshowerEnergy']

    if isEE:
        branches = branches_EE + ["pt", "eta"]
    else:
        branches = branches_EB + ["pt", "eta"]

    if "Electron" in file_name:
        df = root_file['een_analyzer/ElectronTree'].pandas.df(branches, entrystop=entrystop).dropna()
    if "Photon" in file_name:
        df = root_file['een_analyzer/PhotonTree'].pandas.df(branches, entrystop=entrystop).dropna()
    print("Entries in ntuple:")
    print(len(df))
    print(df.shape)


    df.eval("clusertRawEnergyOverE5x5 = clusterRawEnergy/full5x5_e5x5", inplace=True)
    df.eval("w3x3OverE5x5             = full5x5_e3x3/full5x5_e5x5", inplace=True)
    df.eval("eMaxOverE5x5             = full5x5_eMax/full5x5_e5x5", inplace=True)
    df.eval("e2ndOverE5x5             = full5x5_e2nd/full5x5_e5x5", inplace=True)
    df.eval("eTopOverE5x5             = full5x5_eTop/full5x5_e5x5", inplace=True)
    df.eval("eBottomOverE5x5          = full5x5_eBottom/full5x5_e5x5", inplace=True)
    df.eval("eLeftOverE5x5            = full5x5_eLeft/full5x5_e5x5", inplace=True)
    df.eval("eRightOverE5x5           = full5x5_eRight/full5x5_e5x5", inplace=True)
    df.eval("e2x5MaxOverE5x5          = full5x5_e2x5Max/full5x5_e5x5", inplace=True)
    df.eval("e2x5TopOverE5x5          = full5x5_e2x5Top/full5x5_e5x5", inplace=True)
    df.eval("e2x5BottomOverE5x5       = full5x5_e2x5Bottom/full5x5_e5x5", inplace=True)
    df.eval("e2x5LeftOverE5x5         = full5x5_e2x5Left/full5x5_e5x5", inplace=True)
    df.eval("e2x5RightOverE5x5        = full5x5_e2x5Right/full5x5_e5x5", inplace=True)

    if isEE:
        df.eval("preshowerEnergyOverrawEnergy = preshowerEnergy/rawEnergy", inplace=True)

    # The target
    if isEE:
        df.eval("target = ( rawEnergy + preshowerEnergy )/genEnergy", inplace=True)
    else:
        df.eval("target = rawEnergy/genEnergy", inplace=True)

    return df


In [ ]:
# The features
features_EB = [ 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
        'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
        'full5x5_sigmaIphiIphi', 'clusertRawEnergyOverE5x5', 'w3x3OverE5x5',
        'eMaxOverE5x5', 'e2ndOverE5x5', 'eTopOverE5x5', 'eBottomOverE5x5',
        'eLeftOverE5x5', 'eRightOverE5x5', 'e2x5MaxOverE5x5',
        'e2x5TopOverE5x5', 'e2x5BottomOverE5x5', 'e2x5LeftOverE5x5',
        'e2x5RightOverE5x5', 'iEtaSeed', 'iPhiSeed', 'iEtaMod5', 'iPhiMod2',
        'iEtaMod20', 'iPhiMod20','pt','eta','target']

# EE
features_EE = [ 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
        'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
        'full5x5_sigmaIphiIphi', 'clusertRawEnergyOverE5x5', 'w3x3OverE5x5',
        'eMaxOverE5x5', 'e2ndOverE5x5', 'eTopOverE5x5', 'eBottomOverE5x5',
        'eLeftOverE5x5', 'eRightOverE5x5', 'e2x5MaxOverE5x5',
        'e2x5TopOverE5x5', 'e2x5BottomOverE5x5', 'e2x5LeftOverE5x5',
        'e2x5RightOverE5x5', 'iXSeed', 'iYSeed', 'preshowerEnergyOverrawEnergy','pt','eta','target']


file_name = "/scratch/micheli/Electron/perfectIC-lowpt-EB-training.root"


isEE = '-EE-' in file_name

if isEE:
    features = features_EE
else:
    features = features_EB
DF= load_data(file_name, entrystop=1500000,isEE=isEE)



In [ ]:
DF=DF.query('pt>20')

#  Parameter

In [ ]:
out_dir='/work/okiss/SemesterProject_2/xgbo/Energy_Regression_test/'
parameter=pd.read_csv(out_dir+'Electron_perfectIC-lowpt-EB_test2019049/summary.csv')
rangemax=np.array([20,1,15,10,10])
rangemin=np.array([1,0.1,2,0,0])
i=0
j=0
for para in parameter.columns:
    if i==0:
        para0=para
    else:
        print para
        plt.figure()
        plt.plot(parameter[para0],parameter[para],'k.',label='test point')
        plt.plot(parameter[para0].iloc[-1],parameter[para].iloc[-1],'r.',label='final point')
        plt.plot(parameter[para0].iloc[-2],parameter[para].iloc[-2],'m.',label='before last point')
        plt.plot(parameter[para0].iloc[:4],parameter[para].iloc[:4],'b.',label='random points')  
        plt.title(para)
        plt.xlabel('iteration')
        plt.grid()
        
        
        if 4<i<10:
            one =np.linspace(0,np.size(parameter[para0]),1000)
            plt.plot(one,np.ones_like(one)*rangemax[j],'k-',label='maximal range')
            plt.plot(one,np.ones_like(one)*rangemin[j],'k-',label='minimal range')
            j+=1
        #plt.legend()
        plt.savefig(os.path.join(out_dir+'plot', para+'_evolution'+'.png'))
        plt.show()
        plt.figure()
        plt.plot(parameter[para0],parameter[para],'k.',label='test point')
        plt.plot(parameter[para0].iloc[-1],parameter[para].iloc[-1],'r.',label='final point')
        plt.plot(parameter[para0].iloc[-2],parameter[para].iloc[-2],'m.',label='before last point')
        plt.plot(parameter[para0].iloc[:4],parameter[para].iloc[:4],'b.',label='random points') 
        plt.title(para+'_zoom')
        plt.xlabel('iteration')
        plt.grid()
        zoom=(parameter[para].iloc[-1]-parameter[para].iloc[-2])
        zoom=abs(zoom)
        plt.ylim(parameter[para].iloc[-1]-2*zoom,parameter[para].iloc[-1]+1.5*zoom)
        print(zoom)
        plt.legend()
        plt.savefig(os.path.join(out_dir+'plot', para+'_evolution_zoom'+'.png'))
        plt.show()
        
    i+=1

#  Data

In [ ]:
df_test  = load_data(file_name, isEE=isEE, entrystop=1500000)
X_test  = df_test[features_EB]
y_test  = df_test["target"]
xgtest  = xgb.DMatrix(X_test , label=y_test)

xgbo_default = xgb.Booster({'nthread':4}) #init model
#xgbo_default.load_model("Photon_perfectIC-highpt-EB/model_default/model.bin") # load data
xgbo_default.load_model(out_dir+"Electron_perfectIC-lowpt-EB_test20190124/model_default/model.bin")
xgbo_bo = xgb.Booster({'nthread':4}) #init model
#xgbo_bo.load_model("Photon_perfectIC-highpt-EB/model_optimized/model.bin") # load data
xgbo_bo.load_model(out_dir+"Electron_perfectIC-lowpt-EB_test20190124/model_optimized/model.bin") # load data

preds_default = xgbo_default.predict(xgtest)
preds_bo      = xgbo_bo.predict(xgtest)

# preds = 1.

# Etrue / Eraw after applying corrections
z_default = preds_default / (df_test['genEnergy']/df_test['rawEnergy'])
z_bo      = preds_bo / (df_test['genEnergy']/df_test['rawEnergy'])

bins = np.linspace(0.0, 2.0, 200)
bins_zoom = np.linspace(0.9, 1.1, 200)

plt.figure()
plt.hist(df_test['rawEnergy']/df_test['genEnergy'], bins=bins, histtype='step', label="uncorrected")
plt.hist(z_default, bins=bins, histtype='step', label='corrected (default)')
plt.hist(z_bo, bins=bins, histtype='step', label='corrected (optimized)')
plt.legend()
plt.savefig(os.path.join(out_dir+'plot','energy_comparaison'+'.png'))


plt.figure()
plt.hist(df_test['rawEnergy']/df_test['genEnergy'], bins=bins_zoom, histtype='step', label="uncorrected")
plt.hist(z_default, bins=bins_zoom, histtype='step', label='corrected (default)')
plt.hist(z_bo, bins=bins_zoom, histtype='step', label='corrected (optimized)')
plt.legend()
plt.savefig(os.path.join(out_dir+'plot','energy_comparaison_zoom'+'.png'))
plt.show()

In [ ]:
plt.figure()
plt.hist(preds_bo,bins)
plt.show()

In [ ]:
data=pd.DataFrame()
data['eta']=df_test['eta']
data['pt']=df_test['pt']
data['target']=df_test['target']
data['z_default']=z_default
data['z_bo']=z_bo
print(data.columns)

# RMS and effective sigma


In [ ]:
def evaleffrms(hist, bin_edge, c=0.683):
    event=np.sum(hist)
    m=c*event
    a=[]
    b=[]
    for i in range (len(hist)):
        sum=0
        for j in range(i,len(hist)):
            if sum>=m:
                a.append(i)
                b.append(j)
                break
            else:
                sum+=hist[j]
    l=len(a)
    interval=np.zeros((l))
    for i in range(l):
        interval[i]=bin_edge[b[i]]-bin_edge[a[i]]
    effrms=np.min(interval)/2
    return(effrms)
def RMS(hist,bin_edge):
    event=np.sum(hist)
    sum=0
    for i in range(len(hist)):
        sum+=((bin_edge[i+1]+bin_edge[i])/2)**2*hist[i]
    rms=np.sqrt(sum/event)
    return rms
def histsigma(hist,bin_edge):
    N=np.sum(hist)
    bin_edge_middle=np.zeros((len(hist)))
    for i in range (len(hist)):
        bin_edge_middle[i]=(bin_edge[i]+bin_edge[i+1])/2
        
    mean=np.sum(bin_edge_middle*hist)/N
    variance=0
    print(mean)
    for i in range(len(hist)):
        variance+=hist[i]*(bin_edge_middle[i]-mean)**2
    variance=variance/N
    sigma=np.sqrt(variance)
    return sigma

In [ ]:
x=np.random.normal(loc=0,scale=1,size=1000000)
q,w=np.histogram(x,bins=1000)
plt.hist(x,bins=100)
print(evaleffrms(q,w))
print(RMS(q,w))
print(np.mean(q))
print(np.sqrt(np.mean(np.square(q-np.mean(q)))))
print(histsigma(q,w))

# ETA

In [ ]:
n=16
feat=[]
eta=np.zeros((n))
for i in range (n):
    a=1.4-0.2*i
    b=a+0.2
    eta[i]= (a+b)/2
    feat.append(str(a)+'<eta<'+str(b))
i=0
for para in feat:
    rmin = 0.83
    rmax = 1.2
    plt.figure()
    df=data.query(para).astype('double')
    dft=df.query(str(rmin)+'<target<'+str(rmax)).astype('double')
    dfd=df.query(str(rmin)+'<z_default<'+str(rmax)).astype('double')
    dfo=df.query(str(rmin)+'<z_bo<'+str(rmax)).astype('double')
    nbins = 200
    plt.hist(dft['target'],bins=nbins,histtype='step',label='uncorrected',density=0)
    plt.hist(dfd['z_default'],bins=nbins,histtype='step',label='corrected (default)',density=0)
    plt.hist(dfo['z_bo'],bins=nbins,histtype='step',label='corrrected (optimized)',density=0)
    plt.title(para)
    plt.legend(loc=2)
    plt.savefig(os.path.join(out_dir+'plot','split_in_eta_'+para+'.png'))
    
    plt.show()
    #plt.ylim([0,500])
    
    #surface_hist=sum(out[0][:]*np.diff(out[1][:]))
   
   
    
   

In [ ]:
Tipe=['target','z_default','z_bo']
n=16

mean_eta_un=np.zeros((n))
mean_eta_de=np.zeros((n))
mean_eta_bo=np.zeros((n))

mean_err_eta_un=np.zeros((n))
mean_err_eta_de=np.zeros((n))
mean_err_eta_bo=np.zeros((n))

sigma_eta_un=np.zeros((n))
sigma_eta_de=np.zeros((n))
sigma_eta_bo=np.zeros((n))

Eff_sigma_eta_un=np.zeros((n))
Eff_sigma_eta_de=np.zeros((n))
Eff_sigma_eta_bo=np.zeros((n))

rms_eta_un=np.zeros((n))
rms_eta_de=np.zeros((n))
rms_eta_bo=np.zeros((n))

histsigma_eta_un=np.zeros((n))
histsigma_eta_de=np.zeros((n))
histsigma_eta_bo=np.zeros((n))

sigma_err_eta_un=np.zeros((n))
sigma_err_eta_de=np.zeros((n))
sigma_err_eta_bo=np.zeros((n))

feat=[]
eta=np.zeros((n))
nbins=500
for i in range (n):
    a=1.4-0.2*i
    b=a+0.2
    eta[i]= (a+b)/2
    feat.append(str(a)+'<eta<'+str(b))
i=0
for tipe in Tipe:
    for para in feat:
        if tipe=='target':
            rmin = 0.83
            rmax = 1.1 # before 1.1
        else:
            rmin=0.85
            rmax=1.15 #1.2
        df=data.query(para).astype('double')
        df=df[tipe]
        bound_DCB=[rmin, rmax]
        plt.figure()
        hist,bin_edge=np.histogram(df,bins=nbins)
        plt.show()
        hist_mean=np.mean(hist)
        a=RMS(hist,bin_edge)
        b=evaleffrms(hist,bin_edge)
        c=histsigma(hist,bin_edge)
        normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
        binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
   
        for j in range(2):
        #first step
            if j==0: 
                pars_dcb = dict(mean  = 0.97, 
                        fix_mean = False,
                        sigma  = 0.01,
                        fix_sigma = False,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 6,
                        fix_n  = False,
                        alpha2 =1,
                        fix_alpha2 = False,
                        n2     = 4,
                        fix_n2  = False
        
                           )
        
                           
        #optimize parameter withoptimized start parameter
            else:
                pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
    
            minuit = iminuit.Minuit(binned_likelihood,print_level=3, **pars_dcb)
            minuit.migrad() #optimized parameter
     
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
        binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
        if tipe=='target':
            mean_eta_un[i]=minuit.values['mean']
            mean_err_eta_un[i]=minuit.errors['mean']
            sigma_eta_un[i]=minuit.values['sigma']
            sigma_err_eta_un[i]=minuit.errors['sigma']
            rms_eta_un[i]=a
            Eff_sigma_eta_un[i]=b
            histsigma_eta_un[i]=c
        if tipe=='z_default':
            mean_eta_de[i]=minuit.values['mean']
            mean_err_eta_de[i]=minuit.errors['mean']
            sigma_eta_de[i]=minuit.values['sigma']
            sigma_err_eta_de[i]=minuit.errors['sigma'] 
            rms_eta_de[i]=a
            Eff_sigma_eta_de[i]=b
            histsigma_eta_de[i]=c
        if tipe=='z_bo':
            mean_eta_bo[i]=minuit.values['mean']
            mean_err_eta_bo[i]=minuit.errors['mean']
            sigma_eta_bo[i]=minuit.values['sigma']
            sigma_err_eta_bo[i]=minuit.errors['sigma'] 
            rms_eta_bo[i]=a
            Eff_sigma_eta_bo[i]=b
            histsigma_eta_bo[i]=c
        print(para)
    
        i+=1
        if i==16:
            i=0
        
        plt.title(tipe+para)
        plt.savefig(os.path.join(out_dir+'plot',tipe+para+'_fit'+'.png'))
        plt.show()
np.savetxt('eta_parameter.csv', (mean_eta_un,sigma_eta_un,mean_eta_de,sigma_eta_de,mean_eta_bo,sigma_eta_bo), delimiter=',')

In [ ]:
df=data.query('-0.2<eta<0.2').astype('double')
df=df['target']
bound_DCB=[0.8,1.1]
plt.figure()
hist,bin_edge=np.histogram(df,bins=nbins,range=bound_DCB)
plt.hist(df,bins=nbins)
plt.xlim(0.8,1.1)
plt.show()
#print(hist,bin_edge)
a=np.sqrt(np.mean(np.square(hist)))
b=evaleffrms(hist,bin_edge)
#print(a,b)

In [ ]:
plt.figure()
plt.errorbar(eta,mean_eta_un, yerr=mean_err_eta_un, fmt='b.',label='uncorrected',ecolor='black',capsize=1)
plt.errorbar(eta,mean_eta_de, yerr=mean_err_eta_de, fmt='k.',label='default',ecolor='black',capsize=1)
plt.errorbar(eta,mean_eta_bo, yerr=mean_err_eta_bo, fmt='r.',label='optimized',ecolor='black',capsize=1)
plt.title('mean against eta')
plt.xlabel('$\eta$')
plt.ylabel('mean')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','mean_against_eta'+'.png'))

plt.show()

plt.figure()
plt.errorbar(eta,sigma_eta_un, yerr=sigma_err_eta_un,fmt='b.',label='uncorrected',ecolor='black',capsize=1)
plt.errorbar(eta,sigma_eta_de, yerr=sigma_err_eta_de, fmt='k.',label='default',ecolor='black',capsize=1)
plt.errorbar(eta,sigma_eta_bo, yerr=sigma_err_eta_bo, fmt='r.',label='optimized',ecolor='black',capsize=1)
plt.title('sigma against eta')
plt.xlabel('$\eta$')
plt.ylabel('$\sigma$')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','sigma_against_eta'+'.png'))

plt.show()

plt.figure()
plt.plot(eta,Eff_sigma_eta_un,'b.',label='uncorrected')
plt.plot(eta,Eff_sigma_eta_de,'k.',label='default')
plt.plot(eta,Eff_sigma_eta_bo,'r.',label='optimized')
plt.title('Eff_sigma against eta')
plt.xlabel('$\eta$')
plt.ylabel('Effective $\sigma$')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','Eff_sigma_against_eta'+'.png'))

plt.show()

plt.figure()
plt.plot(eta,rms_eta_un,'b.',label='uncorrected')
plt.plot(eta,rms_eta_de,'k.',label='default')
plt.plot(eta,rms_eta_bo,'r.',label='optimized')
plt.title('RMS against eta')
plt.xlabel('$\eta$')
plt.ylabel('RMS')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','RMS_against_eta'+'.png'))


plt.show()

plt.figure()
plt.plot(eta,histsigma_eta_un,'b.',label='uncorrected')
plt.plot(eta,histsigma_eta_de,'k.',label='default')
plt.plot(eta,histsigma_eta_bo,'r.',label='optimized')
plt.title('histsigma against eta')
plt.xlabel('$\eta$')
plt.ylabel('hist sigma')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','histsigma_against_eta'+'.png'))


plt.show()

#  PT

In [ ]:
n=29
pt=np.zeros((n))
mean_pt=np.zeros((n))
err_mean_pt=np.zeros((n))
sigma_pt=np.zeros((n))
err_sigma_pt=np.zeros((n))

feat_pt=[]
for i in range (n):
    a=300-10*i
    b=a+10
    pt[i]= (a+b)/2
    feat_pt.append(str(a)+'<pt<'+str(b))
i=0

for para in feat_pt:
    plt.figure()
    df=DF.query(para).astype('double')
    if i<=23:
        rmin = 0.85
        rmax = 1.15
    elif 23<i<27:
        rmin=0.7
        rmax=1.25
    else:
        rmin=0.6
        rmax=1.4
    plt.figure()
    df=data.query(para).astype('double')
    dft=df.query(str(rmin)+'<target<'+str(rmax)).astype('double')
    dfd=df.query(str(rmin)+'<z_default<'+str(rmax)).astype('double')
    dfo=df.query(str(rmin)+'<z_bo<'+str(rmax)).astype('double')
    nbins = 200
    plt.hist(dft['target'],bins=nbins,histtype='step',label='uncorrected',density=0)
    plt.hist(dfd['z_default'],bins=nbins,histtype='step',label='corrected (default)',density=0)
    plt.hist(dfo['z_bo'],bins=nbins,histtype='step',label='corrrected (optimized)',density=0)
    plt.title(para)
    plt.legend(loc=2)
    plt.savefig(os.path.join(out_dir+'plot','split_in_pt_'+para+'.png'))
    print(i)
    plt.show()
    i+=1

In [ ]:
Tipe=['target','z_default','z_bo']
n=29

mean_pt_un=np.zeros((n))
mean_pt_de=np.zeros((n))
mean_pt_bo=np.zeros((n))

mean_err_pt_un=np.zeros((n))
mean_err_pt_de=np.zeros((n))
mean_err_pt_bo=np.zeros((n))

sigma_pt_un=np.zeros((n))
sigma_pt_de=np.zeros((n))
sigma_pt_bo=np.zeros((n))

Eff_sigma_pt_un=np.zeros((n))
Eff_sigma_pt_de=np.zeros((n))
Eff_sigma_pt_bo=np.zeros((n))

rms_pt_un=np.zeros((n))
rms_pt_de=np.zeros((n))
rms_pt_bo=np.zeros((n))

histsigma_pt_un=np.zeros((n))
histsigma_pt_de=np.zeros((n))
histsigma_pt_bo=np.zeros((n))

sigma_err_pt_un=np.zeros((n))
sigma_err_pt_de=np.zeros((n))
sigma_err_pt_bo=np.zeros((n))

pt=np.zeros((n))
feat_pt=[]
for i in range (n):
    a=300-10*i
    b=a+10
    pt[i]= (a+b)/2
    feat_pt.append(str(a)+'<pt<'+str(b))
nbins=500

In [ ]:
# the fit of the uncorrected can been found in PT_Uncorreceted.ipynb where they are done indivually and then 
#loaded into a csv file
tipe='target'
i=0
for para in feat_pt:
    if 21<i<26:
        rmin=0.78
        rmax=1.07
    elif i>25:
        rmin=0.6
        rmax=1.2
    elif 15<i<23:
        rmin=0.79
        rmax=1.11
    else:
        rmin=0.9
        rmax=1.07
    
    df=data.query(para).astype('double')
    df=df[tipe]
    bound_DCB=[rmin, rmax]
    
    plt.figure()
    hist,bin_edge=np.histogram(df,bins=nbins)
    plt.show()
    hist_mean=np.mean(hist)
    a=RMS(hist,bin_edge)
    b=evaleffrms(hist,bin_edge)
    c=histsigma(hist,bin_edge)
    
    rms_pt_un[i]=a
    Eff_sigma_pt_un[i]=b
    histsigma_pt_un[i]=c
    i+=1

In [ ]:
tipe='z_default'
i=0
for para in feat_pt:
  
    if i>23:
        if i>25 and i!=28:
            rmin=0.6
            rmax=1.45
        elif i==28:
            rmin=0.4
            rmax=1.8
        else:
            rmin=0.8
            rmax=1.2
    
    elif i==14 or i==22:
        rmin=0.86
        rmax=1.1
    else:
        rmin = 0.93
        rmax = 1.07
    
    df=data.query(para).astype('double')
    df=df[tipe]
    bound_DCB=[rmin, rmax]
    plt.figure()
    hist,bin_edge=np.histogram(df,bins=nbins)
    plt.show()
    hist_mean=np.mean(hist)
    a=RMS(hist,bin_edge)
    b=evaleffrms(hist,bin_edge)
    c=histsigma(hist,bin_edge)
    normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
    binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
   
    for j in range(5):
        #first step
        if j==0: 
            pars_dcb = dict(mean  = 1.004, 
                        fix_mean = False,
                        sigma  = 0.015,
                        fix_sigma = True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = False,
                        alpha2 =1,
                        fix_alpha2 = False,
                        n2     = 4,
                        fix_n2  = False
        
                           )
        
        
        elif j==1:
                pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = True,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = True
                        )
                
        elif j==3:
            pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = True,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
                
        #optimize parameter withoptimized start parameter
        else:
            pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
    
        minuit = iminuit.Minuit(binned_likelihood,print_level=3, **pars_dcb)
        minuit.migrad() #optimized parameter
        if i!=7 and i !=9 and i!=22 and i!=28:
            if j==2:
                break
            else:
                pass
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
    binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    if tipe=='target':
        mean_pt_un[i]=minuit.values['mean']
        mean_err_pt_un[i]=minuit.errors['mean']
        sigma_pt_un[i]=minuit.values['sigma']
        sigma_err_pt_un[i]=minuit.errors['sigma']
    if tipe=='z_default':
        mean_pt_de[i]=minuit.values['mean']
        mean_err_pt_de[i]=minuit.errors['mean']
        sigma_pt_de[i]=minuit.values['sigma']
        sigma_err_pt_de[i]=minuit.errors['sigma']
        rms_pt_de[i]=a
        Eff_sigma_pt_de[i]=b
        histsigma_pt_de[i]=c
    if tipe=='z_bo':
        mean_pt_bo[i]=minuit.values['mean']
        mean_err_pt_bo[i]=minuit.errors['mean']
        sigma_pt_bo[i]=minuit.values['sigma']
        sigma_err_pt_bo[i]=minuit.errors['sigma'] 
    print(para)
    print(i)
    i+=1
    if i==29:
        i=0
        
    plt.title(tipe+para)
    plt.savefig(os.path.join(out_dir+'plot',tipe+para+'_fit'+'.png'))
    plt.show()
    mean_err_pt_de[0]=0
    sigma_err_pt_de[0]=0
np.savetxt('pt_parameter_uncorrected.csv', (mean_pt_de,sigma_pt_de), delimiter=',')

In [ ]:
tipe='z_bo'
i=0
for para in feat_pt:
    if i>22:
        if i>25 and i!=28:
            rmin=0.6
            rmax=1.45
        elif i==28:
            rmin=0.4
            rmax=1.8
        else:
            rmin=0.8
            rmax=1.2
    
    elif i==4 or i==5 or 14<=i<=16 or i==19:
        rmin=0.9
        rmax=1.1
    else:
        rmin = 0.94
        rmax = 1.06
    
    df=data.query(para).astype('double')
    df=df[tipe]
    bound_DCB=[rmin, rmax]
    plt.figure()
    hist,bin_edge=np.histogram(df,bins=nbins)
    plt.show()
    hist_mean=np.mean(hist)
    a=RMS(hist,bin_edge)
    b=evaleffrms(hist,bin_edge)
    c=histsigma(hist,bin_edge)
    normalized_DCB = probfit.Normalized(probfit.pdf.doublecrystalball, bound_DCB)
    binned_likelihood = probfit.BinnedLH(normalized_DCB, df, bins=nbins,use_w2=False, bound=bound_DCB)
   
   
    for j in range(5):
        #first step
        if j==0: 
            pars_dcb = dict(mean  = 1.004, 
                        fix_mean = False,
                        sigma  = 0.015,
                        fix_sigma = True,
                        alpha  = 1,
                        fix_alpha = False,
                        n      = 10,
                        fix_n  = False,
                        alpha2 =1,
                        fix_alpha2 = False,
                        n2     = 4,
                        fix_n2  = False
        
                           )
        
        
        elif j==1:
                pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = True,
                        n      = minuit.values['n'],
                        fix_n  = True,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = True,
                        n2     = minuit.values['n2'],
                        fix_n2  = True
                        )
                
        elif j==3:
            pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = True,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = True,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
                
        #optimize parameter withoptimized start parameter
        else:
            pars_dcb=dict(mean  = minuit.values['mean'], 
                        fix_mean = False,
                        sigma  = minuit.values['sigma'],
                        fix_sigma = False,
                        alpha  = minuit.values['alpha'],
                        fix_alpha = False,
                        n      = minuit.values['n'],
                        fix_n  = False,
                        alpha2 = minuit.values['alpha2'],
                        fix_alpha2 = False,
                        n2     = minuit.values['n2'],
                        fix_n2  = False
                        )
       
    
        minuit = iminuit.Minuit(binned_likelihood,print_level=3,limit_mean=(0.9,1.1), **pars_dcb)
        minuit.migrad() #optimized parameter
        if i!=7 and i !=9 and i!=22 and i!=28:
            if j==2:
                break
            else:
                pass
     
    
    #normalized_plot_range = probfit.Normalized(probfit.pdf.doublecrystalball, plot_range)
    #binned_likelihood_plot_range = probfit.BinnedLH(normalized_plot_range, data, bins=nbins, bound=plot_range)
    #binned_likelihood_plot_range.draw(minuit,nfbins=nbins,parmloc=(0.5,0.75))
    
    binned_likelihood.draw(minuit,nfbins=nbins, parmloc=(0.05,0.75))
    if tipe=='target':
        mean_pt_un[i]=minuit.values['mean']
        mean_err_pt_un[i]=minuit.errors['mean']
        sigma_pt_un[i]=minuit.values['sigma']
        sigma_err_pt_un[i]=minuit.errors['sigma']
    if tipe=='z_default':
        mean_pt_de[i]=minuit.values['mean']
        mean_err_pt_de[i]=minuit.errors['mean']
        sigma_pt_de[i]=minuit.values['sigma']
        sigma_err_pt_de[i]=minuit.errors['sigma']       
    if tipe=='z_bo':
        mean_pt_bo[i]=minuit.values['mean']
        mean_err_pt_bo[i]=minuit.errors['mean']
        sigma_pt_bo[i]=minuit.values['sigma']
        sigma_err_pt_bo[i]=minuit.errors['sigma']
        Eff_sigma_pt_bo[i]=b
        rms_pt_bo[i]=a
        histsigma_pt_bo[i]=c
    print(para)
    print(i)
    i+=1
    if i==29:
        i=0
        
    plt.title(tipe+para)
    plt.savefig(os.path.join(out_dir+'plot',tipe+para+'_fit'+'.png'))
    plt.show()
np.savetxt('pt_parameter_bo.csv', (mean_pt_bo,sigma_pt_bo), delimiter=',')

In [ ]:
Target_alone=np.loadtxt('pt_parameter_Uncorrected.csv',delimiter=',')
l=np.size((Target_alone))
print(Target_alone)
mean_pt_un_alone=Target_alone[:l/2]
sigma_pt_un_alone=Target_alone[l/2:]
print(mean_pt_un_alone)
print(sigma_pt_un_alone)


In [ ]:
sigma_err_pt_un[26]=0
mean_err_pt_un[26]=0
plt.figure()
plt.errorbar(pt,mean_pt_un_alone, yerr=mean_err_pt_un, fmt='b.',label='uncorrected',ecolor='black',capsize=1)
plt.errorbar(pt,mean_pt_de, yerr=mean_err_pt_de, fmt='k.',label='default',ecolor='black',capsize=1)
plt.errorbar(pt,mean_pt_bo, yerr=mean_err_pt_bo, fmt='r.',label='optimized',ecolor='black',capsize=1)
plt.title('mean against pt')
plt.xlabel('pt')
plt.ylabel('mean')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','mean_against_pt'+'.png'))
plt.show()

plt.figure()
plt.errorbar(pt,sigma_pt_un_alone, yerr=sigma_err_pt_un, fmt='b.',label='uncorrected',ecolor='black',capsize=1)
plt.errorbar(pt,sigma_pt_de, yerr=sigma_err_pt_de, fmt='k.',label='default',ecolor='black',capsize=1)
plt.errorbar(pt,sigma_pt_bo, yerr=sigma_err_pt_bo, fmt='r.',label='optimized',ecolor='black',capsize=1)
plt.title('sigma against pt')
plt.xlabel('pt')
plt.ylabel('$\sigma$')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','sigma_against_pt'+'.png'))
plt.show()

plt.figure()
plt.plot(pt,Eff_sigma_pt_un,'b.',label='uncorrected')
plt.plot(pt,Eff_sigma_pt_de,'k.',label='default')
plt.plot(pt,Eff_sigma_pt_bo,'r.',label='optimized')
plt.title('Eff_sigma against pt')
plt.xlabel('pt')
plt.ylabel('Effective $\sigma$')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','Eff_sigma_against_pt'+'.png'))

plt.show()

plt.figure()
plt.plot(pt,rms_pt_un,'b.',label='uncorrected')
plt.plot(pt,rms_pt_de,'k.',label='default')
plt.plot(pt,rms_pt_bo,'r.',label='optimized')
plt.title('RMS against pt')
plt.xlabel('pt')
plt.ylabel('RMS')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','RMS_against_pt'+'.png'))


plt.show()


plt.figure()
plt.plot(pt,histsigma_pt_un,'b.',label='uncorrected')
plt.plot(pt,histsigma_pt_de,'k.',label='default')
plt.plot(pt,histsigma_pt_bo,'r.',label='optimized')
plt.title('histsigma against pt')
plt.xlabel('pt')
plt.ylabel('hist sigma')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_dir+'plot','histsigma_against_pt'+'.png'))


plt.show()